In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import fuzzywuzzy
from fuzzywuzzy import fuzz
from  nltk import word_tokenize
import torch.optim as optim
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import io
import utils

D:\Softwares\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
cuda = torch.device("cuda:0")

In [3]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [4]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [5]:
def get_processed_tokens_for_english_tweets():
    f = "english/agr_en_train.csv"
    # preprocessing english tweets.
    #ingesting english csv file
    df = pd.read_csv(f,names = ['source','comment','annotation'],encoding='UTF-8')
    df['comment'] = df.comment.str.strip()   # removing spaces
    comments = np.asarray(df['comment'])    # dividing the dataframe into comments and tags and converting to array
    tags = np.asarray(df['annotation'])
    print((len(comments)))
    print(len(tags))
    stop_words = set(stopwords.words('english'))  #english stop words list
    processed_tokens = []
    for comment in comments:
    # comment = "Also see ....hw ur RSS activist caught in Burkha .... throwing beef in d holy temples...https://www.google.co.in/amp/www.india.com/news/india/burkha-clad-rss-activist-caught-throwing-beef-at-temple-pictures-go-viral-on-facebook-593154/amp/,NAGfacebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING"
        comment = comment.lower()   #lower casing each tweets
        Digit_REMOVAL = re.sub(r'[0-9]+', '',comment) #removal of numbers 
        URL_REMOVAL = re.sub(r"http\S+", "", Digit_REMOVAL) # removal of URLS
        tokenizer = nltk.RegexpTokenizer(r"\w+")   # removal of punctuation and tokenizing
        new_words = tokenizer.tokenize(URL_REMOVAL)
        sentence = []
        for word in new_words:
            if word not in stop_words:           #checking for stop words on each sentence
                sentence.append(word)
        processed_tokens.append(sentence)
    return processed_tokens, tags

In [6]:
#-----------------For hinglish dataset

def get_processed_hindi_tokens():
    Hindi_text  = "hindi/agr_hi_dev.csv"
    df1 = pd.read_csv(Hindi_text,names = ['source','comment','annotation'],encoding='UTF-8')
    df1['comment'] = df1.comment.str.strip()   # removing spaces
    hindi_comments = np.asarray(df1['comment'])    # dividing the dataframe into comments and tags and converting to array
    hindi_tags = np.asarray(df1['annotation'])
    print((hindi_comments[1])) 
    processed_Hindi_tokens = []
    for comment in hindi_comments:
    #   comment = "Also see ....hw ur RSS activist caught in Burkha .... throwing beef in d holy temples...https://www.google.co.in/amp/www.india.com/news/india/burkha-clad-rss-activist-caught-throwing-beef-at-temple-pictures-go-viral-on-facebook-593154/amp/,NAGfacebook_corpus_msr_403402,On the death of 2 jawans in LOC CROSS FIRING"
        comment = comment.lower()   #lower casing each tweets
        Digit_REMOVAL = re.sub(r'[0-9]+', '',comment) #removal of numbers 
        URL_REMOVAL = re.sub(r"http\S+", "", Digit_REMOVAL) # removal of URLS
        Emoji_removal = remove_emoji(URL_REMOVAL)
        if (isEnglish(Emoji_removal) == True):
            Emoji_removal = re.sub(r'[^\w\s]','',Emoji_removal)# removal of punctuation and tokenizing
        processed_Hindi_tokens.append(word_tokenize(Emoji_removal))
    print(processed_Hindi_tokens[0])
    return processed_Hindi_tokens


In [7]:
#-----------Transliteration and translation
def get_transliteration_Hinglish_Hindi_dict():
    transliteration_dict = "transliterations.hi-en.csv"
    t_dict = pd.read_csv(transliteration_dict,names = ['Hinglish','Hindi'],encoding='UTF-8',sep='\t')
    t_dict['Hinglish'] = t_dict['Hinglish'].str.strip()
    t_dict['Hindi'] = t_dict['Hindi'].str.strip()
    print(t_dict)
    t_dict = np.asarray(t_dict)
    print("After NP array")
    print(t_dict)
    return t_dict

#--------------profanity dictionary
def get_profanity_dict():
    profanity_dict = "ProfanityText.txt"
    P_dict = pd.read_csv(profanity_dict,names = ['Hinglish','English'],encoding='UTF-8',sep='\t')
    P_dict['Hinglish'] = P_dict['Hinglish'].str.strip()
    P_dict['English'] = P_dict['English'].str.strip()
    print("Profanity")
    print(P_dict)
    P_dict = np.asarray(P_dict)
    print("After NP array")
    print(P_dict)
    return P_dict

In [8]:
#------------------Translation of hindi text back to english-------
def translate_hindi_to_english():
    Hindi_dict = "Hindi_English_Dict.csv"
    H_dict = pd.read_csv(Hindi_dict,names = ['Hindi','English'],encoding='UTF-8')

    HE_dict_F = "HE_dictionary_functions.csv"
    H_dict_F = pd.read_csv(HE_dict_F,names = ['Hindi','English'],encoding='UTF-8')
    H_dict_F['Hindi'] = H_dict_F['Hindi'].str.strip()
    H_dict_F['English'] = H_dict_F['English'].str.strip()

    H_hindi_F = np.asarray(H_dict_F['Hindi'])
    H_english_F = np.asarray(H_dict_F['English'])

    H_dict['Hindi'] = H_dict['Hindi'].str.strip()
    H_dict['English'] = H_dict['English'].str.strip()

    H_hindi = np.asarray(H_dict['Hindi'])
    H_english = np.asarray(H_dict['English'])
    
    HE_dict = dict(zip(H_hindi,H_english))
    H_dict_F = dict(zip(H_hindi_F,H_english_F))

    EH_dict = {v:k for k, v in HE_dict.items()}
    EH_dict_F = {v:k for k, v in H_dict_F.items()}
    
    return HE_dict, H_dict_F, EH_dict, EH_dict_F

In [9]:
# Convert
def get_translated_hindi_english(HE_dict, H_dict_F):
    for i in range(0,len(processed_Hindi_tokens)):
        #print(i)
        for j in range (0,len(processed_Hindi_tokens[i])):
            Str = processed_Hindi_tokens[i][j]
            if(Str in HE_dict):
                processed_Hindi_tokens[i][j] = HE_dict[Str]
            elif(Str in H_dict_F):
                processed_Hindi_tokens[i][j] = H_dict_F[Str]
    print(processed_Hindi_tokens[0])
    return processed_Hindi_tokens

In [10]:
def get_token_translations(processed_tokens, processed_Hindi_tokens, EH_dict, P_dict, t_dict):
    for i in range(0,len(processed_Hindi_tokens)):
        for j in range (0,len(processed_Hindi_tokens[i])):
            flag = 0
            Str1 = (processed_Hindi_tokens[i][j])
            max_ratio = 60
            max_ratio_P = 75   #needs to be adjusted
            if (Str1 in EH_dict): # check whether the values exists in english dictionary or not.
                continue
            for l in range(0,len(P_dict)):
                Str2 = P_dict[l][0]
                Ratiostr1 = fuzz.ratio(Str1,Str2)
                if (Ratiostr1 >= max_ratio_P):
                    print(Ratiostr1)
                    max_ratio_P = Ratiostr1
                    flag = 1
                    print(processed_Hindi_tokens[i][j])
                    processed_Hindi_tokens[i][j] = P_dict[l][1]
                    print(f"{flag}-{processed_Hindi_tokens[i][j]}") 
                    break
            for p in EH_dict_F:
                Ratiostr1 = fuzz.ratio(Str1,str(p))
                if(Ratiostr1 >= 98):
                    flag = 1
                    break
            if (flag == 1):
                continue
            else:
                for k in range(0,len(t_dict)):
                    Str2 = t_dict[k][0]
                    Ratiostr1 = fuzz.ratio(Str1,Str2)
                    if (Ratiostr1 > max_ratio):
                        max_ratio = Ratiostr1
                        processed_Hindi_tokens[i][j] = t_dict[k][1]
    print(processed_Hindi_tokens[0])
    print(processed_Hindi_tokens[1])
    print(processed_tokens[12])
    return processed_Hindi_tokens

In [25]:
def translate_hindi_back_to_English(processed_Hindi_tokens, HE_dict, H_dict_F):
    for i in range(0,len(processed_Hindi_tokens)):
        print(i)
        for j in range (0,len(processed_Hindi_tokens[i])):
            Str = processed_Hindi_tokens[i][j]
            if(Str in HE_dict):
                processed_Hindi_tokens[i][j] = HE_dict[Str]
            elif(Str in H_dict_F):
                processed_Hindi_tokens[i][j] = H_dict_F[Str]
    return processed_Hindi_tokens

In [11]:
def load_vectors(fname):
    path = str(fname)
    fin = io.open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [12]:
!pip install fasttext
import fasttext
import fasttext.util
ft = fasttext.load_model('cc.en.300.bin')
ft.get_dimension()
fasttext.util.reduce_model(ft, 200)
ft.get_dimension()

200

In [13]:
processed_tokens, tags = get_processed_tokens_for_english_tweets()
processed_Hindi_tokens = get_processed_hindi_tokens()
t_dict = get_transliteration_Hinglish_Hindi_dict()
P_dict = get_profanity_dict()
HE_dict, H_dict_F, EH_dict, EH_dict_F = translate_hindi_to_english()
processed_Hindi_tokens = get_translated_hindi_english(HE_dict, H_dict_F)
processed_Hindi_tokens = get_token_translations(processed_tokens, processed_Hindi_tokens, EH_dict, P_dict, t_dict)

11999
11999
First stage par dus jootey khaye Grover  se
['randtv', 'tumhare', 'najayaz', 'baap', 'is', 'area', 'hai', 'ki', 'waha', 'koi', 'nahi', 'has', 'sakta', 'haraami', 'azad', 'mulk', 'hai', 'sab', 'jagah', 'jayenge']
           Hinglish        Hindi
0         hajagiree       हजगिरी
1          chekaanv        चेकॉव
2        spinagaarn   स्पिनगार्न
3             medal         मेडल
4       chetthinaad    चेत्तिनाद
...             ...          ...
14914          roda         रोडा
14915  shymaleshwor  स्यामलेश्वर
14916           bar          वार
14917       leonard    लियोनार्ड
14918      gurudwar   गुरूद्वारा

[14919 rows x 2 columns]
After NP array
[['hajagiree' 'हजगिरी']
 ['chekaanv' 'चेकॉव']
 ['spinagaarn' 'स्पिनगार्न']
 ...
 ['bar' 'वार']
 ['leonard' 'लियोनार्ड']
 ['gurudwar' 'गुरूद्वारा']]
Profanity
       Hinglish         English
0         badir           idiot
1    badirchand           idiot
2       bakland           idiot
3        bhadva            pimp
4     bhootnika  son 

100
kutton
1-dog
80
bhai
1-buffalo
75
maut
1-masturbate
75
tak
1-balls
89
badi
1-idiot
75
mai
1-son-in-law
89
jaata
1-breed
75
tak
1-balls
80
janta
1-pubic
75
mai
1-son-in-law
77
chalta
1-ketchup
80
shor
1-semen
91
halla
1-faggot
77
jaakar
1-animal
89
hota
1-donkey
75
baat
1-breed
75
jara
1-breed
75
mai
1-son-in-law
75
mai
1-son-in-law
83
rakkhi
1-fly
75
mai
1-son-in-law
75
mai
1-son-in-law
75
aata
1-breed
89
jaata
1-breed
75
baat
1-breed
80
gandmarane
1-ass fuck
91
hathi
1-elephant
80
chintiya
1-fucker
91
hathi
1-elephant
75
chua
1-shut up
91
chhoot
1-pussy
83
bahais
1-buffalo
80
ladne
1-dick
75
usi
1-ass
80
pahle
1-idiot
75
hui
1-pussy
75
usi
1-ass
75
khaa
1-cut
75
dono
1-drown
75
math
1-masturbate
75
kafi
1-nigger
86
bura
1-pussy
75
uda
1-dick
77
bahen
1-sister fucker
80
loude
1-dick
80
pahle
1-idiot
75
chor
1-fuck
75
sahi
1-stinking
100
dum
1-tail
75
dono
1-drown
77
bahane
1-sister’s
80
janti
1-pubic
75
gha
1-donkey
89
baba
1-boobs
75
sahi
1-stinking
80
hain
1-buffalo
80
shaadi
1-s

80
ladai
1-dick
89
sala
1-sister’s husband
80
fatti
1-shit
89
sala
1-sister’s husband
100
dum
1-tail
80
kiya
1-bitch
89
hota
1-donkey
80
kiya
1-bitch
80
bhai
1-buffalo
86
maa
1-mother’s
75
chod
1-pussy
80
saale
1-sister’s husband
75
kaha
1-cut
83
ghalti
1-hooker
86
jaa
1-breed
75
beheading
1-sister
80
keema
1-bug
75
maar
1-mother’s
75
lalu
1-idiot
77
chara
1-cot
75
kahi
1-nigger
92
harami
1-bastard
75
sahi
1-stinking
75
kaha
1-cut
86
kal
1-nigger
75
the
1-idiot
75
krte
1-dog
80
bhai
1-buffalo
77
chutiye
1-fucker
77
gudsaala
1-sister’s husband
100
toota
1-broken
75
kabi
1-nigger
80
bhai
1-buffalo
75
haa
1-cunt
75
baat
1-breed
75
ker
1-bug
86
kai
1-nigger
77
khatam
1-bug
75
nay
1-eggs
80
kutay
1-dog
89
madachood
1-motherfucker
75
ker
1-bug
80
jamin
1-son-in-law
80
andar
1-semi-dick
75
bhej
1-semen
75
mai
1-son-in-law
75
kuto
1-dog
89
kuta
1-dog
80
banna
1-semi-dick
80
bhai
1-buffalo
80
kiya
1-bitch
80
aukaat
1-cut
75
wali
1-nigger
77
aadat
1-ass
86
muh
1-masturbate
75
mada
1-mother’s
75


86
kal
1-nigger
80
pahle
1-idiot
80
baar
1-gossip
75
the
1-idiot
83
chtiya
1-fucker
75
gau
1-asshole
75
gau
1-asshole
80
hath
1-elephant
75
baat
1-breed
80
jante
1-pubic
75
asli
1-nigger
80
bhai
1-buffalo
75
baat
1-breed
75
krte
1-dog
83
chutya
1-fucker
100
kutta
1-dog
75
kara
1-cut
80
badha
1-semi-dick
75
akal
1-mother’s
80
baka
1-gossip
75
ati
1-shit
75
the
1-idiot
80
bhai
1-buffalo
86
maa
1-mother’s
77
bhosada
1-pimp
86
maa
1-mother’s
91
bhain
1-buffalo
77
chinta
1-whore
86
maa
1-mother’s
75
bhee
1-semen
75
hoo
1-pussy
75
gai
1-filthy
75
hui
1-pussy
89
bana
1-semi-dick
75
han
1-pubic
75
han
1-pubic
75
chor
1-fuck
75
han
1-pubic
75
han
1-pubic
75
haa
1-cunt
80
padhe
1-idiot
92
harami
1-bastard
80
karta
1-dick cut
75
iss
1-bug
80
baar
1-gossip
80
bhai
1-buffalo
91
saaala
1-sister’s husband
100
tatti
1-shit
80
janta
1-pubic
80
bahir
1-idiot
100
tatti
1-shit
100
saala
1-sister’s husband
100
bc
1-sister fucker
75
sahi
1-stinking
89
baba
1-boobs
83
gaaand
1-asshole
83
bhoosa
1-pussy
80
bh

75
aato
1-breed
100
bc
1-sister fucker
80
pahle
1-idiot
80
andar
1-semi-dick
75
baat
1-breed
75
krte
1-dog
80
hain
1-buffalo
80
tmc
1-mother fucker
77
virodhi
1-condom.
89
sala
1-sister’s husband
75
krte
1-dog
80
katha
1-dick cut
75
padi
1-stinking
89
hota
1-donkey
89
hota
1-donkey
75
vhod
1-fuck
75
jati
1-breed
77
chatne
1-ketchup
75
jai
1-son-in-law
75
baat
1-breed
75
kaam
1-cut
75
kaam
1-cut
75
utha
1-masturbate
91
ghadha
1-donkey
80
kiya
1-bitch
80
kiya
1-bitch
80
bhai
1-buffalo
75
baat
1-breed
80
hain
1-buffalo
75
mai
1-son-in-law
77
chahta
1-ketchup
86
kal
1-nigger
75
kee
1-bug
75
kha
1-donkey
80
badla
1-semi-dick
80
hain
1-buffalo
80
hain
1-buffalo
80
aram
1-bastard
80
chhodo
1-fuck
100
neech
1-low caste
75
kaam
1-cut
91
chuni
1-clit
80
halat
1-cunt
75
jate
1-breed
80
hain
1-buffalo
75
haat
1-breed
80
kiya
1-bitch
75
baat
1-breed
92
khatma
1-bug
75
sahi
1-stinking
75
kaha
1-cut
75
jait
1-breed
75
waat
1-breed
83
china
1-whore
80
choro
1-pussy
77
gussega
1-fuck
80
thode
1-fuck
77

...

80
janta
1-pubic
83
gandagi
1-filthy
83
maarne
1-dead
80
bhai
1-buffalo
77
bharne
1-pimp
80
bhai
1-buffalo
75
baat
1-breed
91
puchi
1-pussy
77
sharam
1-bastard
77
bachalo
1-sister fucker
80
bade
1-pimp
75
sahi
1-stinking
80
bohot
1-ghost
89
bana
1-semi-dick
75
baat
1-breed
75
baat
1-breed
75
baat
1-breed
75
aati
1-breed
75
jata
1-breed
75
usi
1-ass
80
ladte
1-dick
80
ladte
1-dick
80
ladte
1-dick
80
ladte
1-dick
80
ladte
1-dick
80
ladte
1-dick
75
gali
1-nigger
75
gali
1-nigger
80
bhai
1-buffalo
83
jihaad
1-terrorist
89
jaate
1-breed
80
bade
1-pimp
80
aram
1-bastard
75
gali
1-nigger
100
banda
1-semi-dick
80
bhai
1-buffalo
80
janta
1-pubic
77
virodhi
1-condom.
80
bhala
1-boobs
80
hain
1-buffalo
80
bhai
1-buffalo
75
rahatha
1-night fuck
75
rahatha
1-night fuck
80
madarchod
1-mother fucker
80
hijda
1-transsexual
75
mara
1-mother’s
89
hota
1-donkey
86
maa
1-mother’s
77
chudate
1-fucking
75
ate
1-idiot
80
kiya
1-bitch
75
the
1-idiot
75
dono
1-drown
89
hota
1-donkey
75
krte
1-dog
92
aatnki
1-t

75
kha
1-donkey
75
maza
1-mother’s
83
kamal
1-bug
77
chinta
1-whore
75
dna
1-stick
89
atankwadi
1-terrorist
75
gaa
1-ass
75
muta
1-masturbate
86
kal
1-nigger
75
dono
1-drown
75
sahi
1-stinking
75
ata
1-dick cut
77
dharam
1-bastard
77
dharam
1-bastard
89
saal
1-sister’s husband
75
kara
1-cut
77
bhaiya
1-snack
75
baat
1-breed
80
banao
1-semi-dick
80
andar
1-semi-dick
75
ghus
1-sucker
75
jai
1-son-in-law
75
hond
1-fuck
94
bhenchod
1-sister fucker
86
muh
1-masturbate
75
utha
1-masturbate
75
ghus
1-sucker
80
bande
1-semi-dick
75
jate
1-breed
80
hain
1-buffalo
80
gand
1-asshole
75
ghus
1-sucker
80
badla
1-semi-dick
75
admi
1-stinking
75
ati
1-shit
84
madrchod
1-mother fucker
75
ati
1-shit
78
bhonsdi
1-fucker
77
chutiyo
1-fucker
75
krte
1-dog
100
kutta
1-dog
80
chate
1-pussy
75
utha
1-masturbate
80
hara
1-bastard
75
jai
1-son-in-law
75
jai
1-son-in-law
75
sahi
1-stinking
80
bhai
1-buffalo
80
rajan
1-whore
83
birodh
1-condom.
80
karta
1-dick cut
86
muh
1-masturbate
75
utha
1-masturbate
77
chal

86
maa
1-mother’s
77
choddege
1-fuck
80
kiya
1-bitch
80
katte
1-dog
75
kaya
1-cut
75
kaya
1-cut
75
jata
1-breed
75
kaha
1-cut
75
baat
1-breed
80
hain
1-buffalo
80
hain
1-buffalo
75
usi
1-ass
75
baat
1-breed
89
bana
1-semi-dick
75
kaam
1-cut
80
kiya
1-bitch
75
madarso
1-motherfucker
75
baat
1-breed
86
maa
1-mother’s
75
chut
1-shut up
92
chutiya
1-fucker
89
bana
1-semi-dick
80
kiya
1-bitch
80
bhai
1-buffalo
83
chata
1-ketchup
75
kadi
1-stinking
80
br
1-pussy
80
br
1-pussy
80
br
1-pussy
80
br
1-pussy
80
br
1-pussy
80
br
1-pussy
100
kutta
1-dog
75
mai
1-son-in-law
77
nirdosh
1-condom.
75
mai
1-son-in-law
75
kaa
1-migga
86
maderchode
1-mother fucker
89
sala
1-sister’s husband
86
maderchode
1-mother fucker
75
mai
1-son-in-law
80
ladai
1-dick
75
mai
1-son-in-law
75
mai
1-son-in-law
89
dada
1-stick
75
mara
1-mother’s
80
lundia
1-dick
77
chakka
1-idiot
80
bhai
1-buffalo
75
ata
1-dick cut
75
okat
1-cut
75
kha
1-donkey
80
salay
1-sister’s husband
75
cmnt
1-pussy
75
mai
1-son-in-law
80
gand
1-assh

75
wali
1-nigger
86
bura
1-pussy
80
bhala
1-boobs
75
baat
1-breed
83
maregy
1-dead
80
bhai
1-buffalo
75
the
1-idiot
75
chod
1-pussy
75
kaam
1-cut
77
khamba
1-bug
100
bc
1-sister fucker
83
maarne
1-dead
75
unta
1-pussy
75
saat
1-breed
100
chup
1-shut up
89
jaati
1-breed
89
jaate
1-breed
80
hain
1-buffalo
92
chutiya
1-fucker
86
muh
1-masturbate
75
baat
1-breed
75
mai
1-son-in-law
77
chahta
1-ketchup
89
aantkvadi
1-terrorist
80
padta
1-fat bitch
89
aantkvadi
1-terrorist
89
aantkvadi
1-terrorist
89
sala
1-sister’s husband
80
padai
1-fat bitch
89
hota
1-donkey
75
mai
1-son-in-law
75
aata
1-breed
75
mai
1-son-in-law
75
mai
1-son-in-law
80
karta
1-dick cut
75
maaa
1-mother’s
80
pahle
1-idiot
75
dan
1-stick
75
chor
1-fuck
80
pahle
1-idiot
77
chadhao
1-fuck
80
kamai
1-son-in-law
89
rndi
1-hooker
89
hota
1-donkey
75
bna
1-semi-dick
100
ullu
1-idiot
80
choro
1-pussy
77
chaahte
1-licker
89
bana
1-semi-dick
80
badan
1-semi-dick
75
phod
1-fuck
80
dalna
1-stick
78
dikhana
1-whore house
75
mout
1-mast

86
burf
1-pussy
75
jata
1-breed
80
hath
1-elephant
80
hath
1-elephant
80
janta
1-pubic
80
loota
1-broken
80
hain
1-buffalo
75
burra
1-pussy
89
bana
1-semi-dick
80
hath
1-elephant
100
bc
1-sister fucker
80
bombu
1-bamboo
75
jong
1-bug
80
kiya
1-bitch
86
najayaj
1-illegitimate
77
chara
1-cot
80
paidaish
1-badborn
80
bhai
1-buffalo
80
bhai
1-buffalo
80
paath
1-idiot
75
mai
1-son-in-law
80
kutte
1-dog
77
chaiye
1-snack
80
madarchod
1-mother fucker
83
dhandha
1-stick
80
kiya
1-bitch
75
sahi
1-stinking
75
kaha
1-cut
89
hota
1-donkey
80
kahta
1-donkey
75
kaam
1-cut
80
alawa
1-dick
89
hota
1-donkey
89
hota
1-donkey
75
ker
1-bug
100
gaandu
1-asshole
80
saalo
1-sister’s husband
75
sal
1-sister’s husband
75
baat
1-breed
80
madarchod
1-mother fucker
91
ghuss
1-fuck
75
maar
1-mother’s
75
kaat
1-breed
80
sawal
1-sister’s husband
75
iss
1-bug
100
chup
1-shut up
75
maan
1-mother’s
100
chup
1-shut up
75
katy
1-cut
75
jaty
1-breed
80
khtay
1-donkey
75
ghus
1-sucker
80
hain
1-buffalo
80
andar
1-semi-dick

In [27]:
processed_Hindi_tokens = translate_hindi_back_to_English(processed_Hindi_tokens, HE_dict, H_dict_F)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410


In [28]:
processed_Hindi_tokens

[['रॉड',
  'तुम्हारे',
  'illegitimate',
  'father',
  'have',
  'area',
  'have',
  'की',
  'स्वाहा',
  'की',
  'other than',
  'हैट्स',
  'सांता',
  'bastard',
  'आज़ाद',
  'माल्क',
  'have',
  'sub',
  'जयगढ़',
  'जाएगा'],
 ['first', 'stage', 'par', 'डेटस्कलैंड', 'टूहे', 'credit', 'ओवर', 'against'],
 ['समन',
  'yet',
  'आमिराह',
  'की',
  'कुंती',
  'Mobile',
  'release',
  'in the very act of',
  'who',
  'windstorm',
  'me',
  'dub',
  'गए',
  'बिकल',
  'भाटकर',
  'media'],
 ['वीक',
  'yet',
  'adorer',
  'inferno',
  'गया',
  'अम्बे',
  'अमेरिकन',
  'of',
  'naif',
  'only',
  'की',
  'इंडियन',
  'K',
  'कर्सन',
  'those',
  'unemployed',
  'buffalo',
  'Muslim',
  'lnk',
  'हिन्दू',
  'lnk',
  'or',
  'christian',
  'lnk',
  'unseeing',
  'on',
  'धरम',
  'K',
  'legislation',
  'yet',
  'कच्छ',
  'डिक',
  'hi',
  'other than'],
 ['beta',
  'to',
  'टू',
  'she',
  'father',
  'of',
  'have',
  'permission',
  'to',
  'टू',
  'value',
  'only',
  'लीयर',
  'ab',
  'मुकद्दर',
  'ma

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long).to(cuda)

def prepare_sequence_tags(seq, to_ix):
    idxs = to_ix[seq]
    idxs = torch.tensor(idxs, dtype=torch.long).to(cuda)
    idxs = idxs.view(1)
    return idxs

In [ ]:
def sentence_to_padded_sentence(sentence,word_to_ix):
    
    # map sentences to vocab
    sentence =  [[word_to_ix[word] for word in sent] for sent in sentence]
    # sentence now looks like:  
    # [[1, 2, 3, 4, 5, 6, 7], [8, 8], [7, 9]]
    sentence_lengths = [len(sent) for sent in sentence]
    pad_token = word_to_ix['<PAD>']
    longest_sent = max(sentence_lengths)
    batch_size = len(sentence)
    padded_sentence = np.ones((batch_size, longest_sent)) * pad_token
    for i, x_len in enumerate(sentence_lengths):
        sequence = sentence[i]
        padded_sentence[i, 0:x_len] = sequence[:x_len]
  
    return padded_sentence

In [ ]:
training_data = utils.substitute_with_UNK(processed_tokens,1)
testing_data = utils.substitute_with_UNK_for_TEST(processed_tokens,word_to_ix)
print(len(training_data))

In [ ]:
word_to_ix = {}
ix_to_word = {}
tag_to_ix = {}
ix_to_tag = {}
word_to_ix = {"<PAD>":0}
for sent in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            ix_to_word[word_to_ix[word]] = word
for tag in tags:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)
        ix_to_tag[tag_to_ix[tag]] = tag

sentence= []
for sent in training_data:
     sentence.append(sent[:50])

In [ ]:
test_sentence = []
for sent in testing_data:
     test_sentence.append(sent[:50])

In [ ]:
padded_sentence = sentence_to_padded_sentence(sentence, word_to_ix)
test_padded_sentence = sentence_to_padded_sentence(test_sentence, word_to_ix)

In [ ]:
class MIMCT(nn.Module):   
    def __init__(self,input_channel,vocab_size,word_to_ix,output_channel,embedding_dim,hidden_dim,kernel_size,feature_linear):
        super(MIMCT, self).__init__()
        self.CNN_Layers = nn.Sequential( 
            nn.Conv1d(input_channel, output_channel,kernel_size[0], stride=1),
            nn.Conv1d(input_channel, output_channel, kernel_size[1], stride=1),
            nn.Conv1d(input_channel, output_channel, kernel_size[2], stride=1),
            nn.Flatten(),nn.Dropout(p=0.25),
            nn.Linear(feature_linear, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
            nn.Softmax()
            )
        
        
        #create LSTM.
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim,3)
        self.dropout = nn.Dropout(p=0.20)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=1)
        self.linear = nn.Linear(50+1,3)
    def forward(self,x):
      #  y = self.LSTM_Layers(x)
        embeds = self.word_embeddings(x)
        
        embeds_cnn = embeds.view(1,embeds.size(0),embeds.size(1))
        cnn_output = self.CNN_Layers(embeds_cnn)
        
        lstm_out, _ = self.lstm(embeds.view(len(x), 1, -1))
        lstm_out= self.dropout(lstm_out)
        tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        
        lstm_output = self.sigmoid(tag_space)
        #concat the outputs the compile layer with categorical cross-entropy the loss function,
        lstm_output = lstm_output.view(lstm_output.size(0),-1)
        cnn_output = cnn_output.view(cnn_output.size(0),-1)
        X = torch.cat((lstm_output,cnn_output))
        X = X.view(1,X.size(0),X.size(1))
        X = self.maxpool(X)
        X = self.linear(X.view(X.size(2), -1))
        X = self.softmax(X)
        print(X)
        return X

In [ ]:
batch_size = 1
input_channel = 10 #vocab size
vocab_size = len(word_to_ix) 
embedding_dim = 200 
output_channel = 10
kernel_size = [20,15,10]
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel

In [ ]:
training_data = utils.substitute_with_UNK(processed_tokens,1)
print(len(training_data))

word_to_ix = {}
ix_to_word = {}
tag_to_ix = {}
ix_to_tag = {}
for sent in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            ix_to_word[word_to_ix[word]] = word
for tag in tags:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)
        ix_to_tag[tag_to_ix[tag]] = tag

In [ ]:
batch_size = 1
input_channel = 50 #vocab size
vocab_size = len(word_to_ix) 
embedding_dim = 200 
output_channel = 50
kernel_size = [20,15,10]
Feature_layer1 = embedding_dim - kernel_size[0] + 1
Feature_layer2 = Feature_layer1 - kernel_size[1] + 1
Feature_layer3 = Feature_layer2 - kernel_size[2] + 1
feature_linear = Feature_layer3 * input_channel

#Parameters for LSTM
hidden_dim = 128
dropout = 0.25, 
#recurrent_dropout = 0.3

model = MIMCT(input_channel,vocab_size,word_to_ix,output_channel,embedding_dim,hidden_dim,kernel_size,feature_linear).to(cuda)
loss_function = nn.CrossEntropyLoss()
#Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
original_data = training_data
training_data = padded_sentence
sentence1 = training_data[0]

print(len(tags))

In [ ]:
for epoch in range(10):  # running for 20 epoch
    print(f"Starting epoch {epoch}...")
    #for sentence in training_data:
    for index,sentence in enumerate(training_data):
        model.zero_grad()
        targets = tags[index]
        sentence_in=torch.tensor(sentence, dtype=torch.long)
        
        targets = prepare_sequence_tags(targets, tag_to_ix)

        tag_scores = model(sentence_in)
 
        loss = loss_function(tag_scores, targets)
        print(loss)
        loss.backward()
        optimizer.step()

In [24]:
processed_Hindi_tokens

import csv

with open("hindi_tokens_list.csv", "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(processed_Hindi_tokens)

In [ ]:
testing_data = test_padded_sentence
with torch.no_grad():
	# this will be the file to write the outputs
    with open("mymodel_output.txt", 'w',encoding='UTF-8') as op:
        for instance in testing_data:
            # Convert the test sentence into a word ID tensor
            test_sentence_in=torch.tensor(instance, dtype=torch.long)

            tag_scores = model(test_sentence_in)

            # Find the tag with the highest probability in each position
            outputs = [int(np.argmax(ts)) for ts in tag_scores.detach().numpy()]
            # Prepare the output to be written in the same format as the test file (word|tag)
            formatted_output = ix_to_tag[outputs[0]]
            # Write the output
            op.write(formatted_output + '\n')
            
        print(i)
        print(len(test_data))